**Understanding the problem statement:**

Based on the past weather data, create a model which takes the current weather information and predict the daily summary

**Understanding the Dataset:**

#

**More about the labels:**

> There are 207 unique labels in training dataset

> There are 32 unique labels in test dataset

> Out of those two datasets 7 labels from test hasn't occured in training dataset

> 75% of labels have count below 168 and 64 for train and test dataset

>

Drizzle until morning. 24

Rain until afternoon. 24

Light rain overnight. 24

Light rain starting overnight. 24

Rain until morning. 24

Rain throughout the day. 24

Drizzle starting in the evening. 24

In [ ]:
!pip -q install ../input/sklearn-0-24-2/scikit_learn-0.24.2-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from matplotlib import pyplot as plt

In [ ]:
weather=pd.read_csv('../input/iitkanpur/train.csv')
weather.head()

In [ ]:
weather.shape

In [ ]:
weather.info()

In [ ]:
fig, (ax1, ax2,ax3) = plt.subplots(1,3, figsize=(20,6))

weather['Summary'].value_counts().plot(kind='bar', facecolor='y',ax=ax1,title='Summary Plot')
weather['Precip Type'].value_counts().plot(kind='bar', facecolor='y',ax=ax2,title='PrecipType Plot')
weather['Cloud Cover'].value_counts().plot(kind='bar', facecolor='y',ax=ax3,title='CloudCover Plot')
plt.show()

In [ ]:
weather['Daily Summary'].value_counts()

In [ ]:
weather.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

weather.hist(bins=50, figsize=(20,15))
#save_fig("attribute_histogram_plots")
plt.show()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
oe.fit(np.array(weather['Daily Summary']).reshape(-1,1))

In [ ]:
weather['Daily Encoded']=oe.transform(np.array(weather['Daily Summary']).reshape(-1,1))
oe.categories_[0][:10]

In [ ]:
corr_matrix=weather.corr()
corr_matrix['Daily Encoded'].sort_values(ascending=False)

In [ ]:
import seaborn as sns

%matplotlib inline
plt.figure(figsize=(10,5))
sns.heatmap(weather.isnull(),yticklabels=False,cbar=False)

plt.show()

In [ ]:
weather=weather.dropna()
weather.shape

In [ ]:
weather.duplicated().sum()

In [ ]:
weather=weather.drop_duplicates()
weather.shape

In [ ]:
tmp=weather['Daily Encoded'].value_counts()
tmp.describe()

In [ ]:
weather.columns

In [ ]:
y=weather['Daily Encoded'].copy()
weather.drop(['Formatted Date','Daily Summary','Daily Encoded'],inplace=True,axis=1)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

num_attribs = ['Temperature (C)','Apparent Temperature (C)','Humidity','Wind Speed (km/h)','Wind Bearing (degrees)',
              'Visibility (km)','Pressure (millibars)']
cat_attribs = ['Summary','Precip Type']

full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), num_attribs),
        ("cat", OneHotEncoder(handle_unknown='ignore'), cat_attribs),
    ])

weather_prepared = full_pipeline.fit_transform(weather)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(weather_prepared,y, test_size=0.2, random_state=42,stratify=y)

In [ ]:
X_train.shape,X_val.shape

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(class_weight='balanced')
clf.fit(X_train,y_train)

In [ ]:
clf.score(X_train,y_train)

In [ ]:
testdata=pd.read_csv('../input/iitkanpur/test.csv')
testdata.head()

In [ ]:
testdata['Daily Encoded']=oe.transform(np.array(testdata['Daily Summary']).reshape(-1,1))

y_test=testdata['Daily Encoded']

testdata.drop(['Formatted Date','Daily Summary','Daily Encoded'],inplace=True,axis=1)
X_test=full_pipeline.transform(testdata)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
evaluation=pd.read_csv('../input/iitkanpur/evaluate.csv')
evaluation.head()

In [ ]:
dates=evaluation['Formatted Date'].copy()
evaluation.drop('Formatted Date',axis=1,inplace=True)
X_eval=full_pipeline.transform(evaluation)

In [ ]:
labels=clf.predict(X_eval)

In [ ]:
submission={}
submission['Formatted Date']=dates
submission['Daily Summary']=[i[0] for i in oe.inverse_transform(np.array(labels).reshape(-1,1))]
submission=pd.DataFrame(submission)
submission.head()

In [ ]:
submission.to_csv('prediction_output.csv',index=False)